In [1]:
from unsloth import FastLanguageModel
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
import csv

/vol/bitbucket/kza23/msc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
DIALECT_NAME = "db_dialect"

csv.register_dialect(
    DIALECT_NAME,
    delimiter=",",
    quoting=csv.QUOTE_MINIMAL,
    escapechar="\\",
)

In [3]:
max_seq_length = 20  # Choose any! We auto support RoPE Scaling internally!
dtype = (
    None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
)
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA A30 MIG 2g.12gb. Max memory: 11.688 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
dataset_vulnerable = load_dataset(
    "msc-smart-contract-audition/vulnerable-functions-base",
    split="train",
    escapechar="\\",
)

Generating train split: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1972/1972 [00:00<00:00, 4632.02 examples/s]


In [5]:
fix_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
The below is a code blocks which might have improper formatting. The task is to fix the formatting and remove comments which do not describe the code. Apart from formatting all of the code needs to remain the same.

1. Ensure that the code is properly indented.
2. Ensure that there are no leading tabs - the outer most block should start with no whitespace infront.
3. If a line is too long break it in multiple lines.
4. Keep all the code the same even if it seems unnecessary in the context.
5. Remove comments which contain no information about the code.
6. Remove any comments which contain URLs.
7. Do not add any backticks as this will be done automatically. If you do it will break the formatting. DO NOT ADD BACKTICKS.

Do not explain your changes at the end just output the formatted code block.
<|start_header_id|>user<|end_header_id|>
Code block:
{}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
Formatted code block:
"""



In [6]:
def extract_codeblocks(description):
    codeblocks = []
    codeblock = ""

    writing = False

    for line in description.split("\n"):
        if line.strip().startswith("```"):
            if writing:
                codeblocks.append(codeblock)
                yield codeblock
                codeblock = ""
                writing = False
            else:
                writing = True
            continue

        if writing:
            codeblock += line + "\n"

In [7]:
# Convert to pandas DataFrame and drop 534 (too long)
descriptions = dataset_vulnerable.to_pandas().drop(index=534)['description'].apply(lambda row: row.replace("\\n", "\n"))

In [26]:
chunk_len = 100
current_chunk = 0
start = current_chunk * chunk_len
end = (current_chunk + 1) * chunk_len

with open(f"cleaned-up-code-{current_chunk+1}.csv", "w", newline="") as f:
    writer = csv.writer(f, dialect=DIALECT_NAME)

    # Write the header row
    writer.writerow(["code"])

    for description in tqdm(descriptions[start:end], total=chunk_len):
        formatted_blocks = []
        for codeblock in extract_codeblocks(description):
            prompt = fix_template.format(codeblock)
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to("cuda")
            output_tokens = model.generate(
                **inputs, max_new_tokens=512, pad_token_id=tokenizer.pad_token_id
            )
            decoded_output = tokenizer.decode(
                output_tokens[0],
                skip_special_tokens=True,
                pad_token_id=tokenizer.pad_token_id,
            )

            raw_output = decoded_output.split("Formatted code block:\n")[1].strip()
            formatted_blocks.append(
                f"```\n{raw_output}\n```"
            )
        codeblocks = "\n".join(formatted_blocks).replace("\n", "\\n")
        writer.writerow([codeblocks])


  0%|                                                                      | 0/100 [00:00<?, ?it/s]

```
(
    netPnLE36,
    lenderProfitUSDValueE36,
    borrowTotalUSDValueE36,
    positionOpenUSDValueE36,
    sharingProfitTokenAmts
) = calcProfitInfo(_positionManager, _user, _posId);

// 2. add liquidation premium to the shared profit amounts
uint lenderLiquidationPremiumBPS = IConfig(config).lenderLiquidatePremiumBPS();

for (uint i = 0; i < sharingProfitTokenAmts.length; i++) {
    sharingProfitTokenAmts[i] += (pos.openTokenInfos[i].borrowAmt * lenderLiquidationPremiumBPS) / BPS;
}
```
```
function _shareProfitsAndRepayAllDebts(
    address _positionManager,
    address _posOwner,
    uint _posId,
    int _netPnLE36,
    uint[] memory _shareProfitAmts,
    address[] memory _tokens,
    OpenTokenInfo[] memory _openTokenInfos
) internal {
    // 0. load states
    address _lendingProxy = lendingProxy;

    // 1. if net pnl is positive, share profits to lending proxy
    if (_netPnLE36 > 0) {
        for (uint i = 0; i < _shareProfitAmts.length; i++) {
            if (_shareProfitAm

  0%|                                                                      | 0/100 [00:18<?, ?it/s]


KeyboardInterrupt: 

In [64]:
fix_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
The below is a code blocks which might have improper formatting. The task is to fix the formatting and remove comments which do not describe the code. Apart from formatting all of the code needs to remain the same.
1. Ensure that the code is properly indented.
2. Ensure that there are no leading tabs - the outer most block should start with no whitespace infront.
3. If a line is too long break it in multiple lines.
4. Keep all the code the same even if it seems unnecessary in the context.
5. Remove comments which contain no information about the code.
6. Remove any comments which contain URLs.

Do not explain your changes at the end just output the formatted code block.
<|start_header_id|>user<|end_header_id|>
Code block:
```
    // File input.sol:12
            uint inputTotalUSDValueE36;
                for (uint i; i < openTokenInfos.length; ) {
                 inputTotalUSDValueE36 += openTokenInfos[i].inputAmt * tokenPriceE36s[i];
                      borrowTotalUSDValueE36 += openTokenInfos[i].borrowAmt * tokenPriceE36s[i];
                 unchecked {
            ++i; // See https://github.com/MSc-Smart-Contract-Audition/data-scraper/blob/main/verified/src/compile_contracts.py
                }
            }
                // 1.3 calculate net pnl (including strategy users & borrow profit)
            positionOpenUSDValueE36 = inputTotalUSDValueE36 + borrowTotalUSDValueE36;
            netPnLE36 = positionCurUSDValueE36.toInt256() - positionOpenUSDValueE36.toInt256();
```
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
Formatted code block:
```
"""


inputs = tokenizer(fix_template, return_tensors="pt", truncation=True).to("cuda")
output_tokens = model.generate(
    **inputs, max_new_tokens=512, pad_token_id=tokenizer.pad_token_id
)
decoded_output = tokenizer.decode(
    output_tokens[0], skip_special_tokens=True, pad_token_id=tokenizer.pad_token_id
)


print(decoded_output.split("Formatted code block:\n")[1].strip())

```
uint inputTotalUSDValueE36;

for (uint i; i < openTokenInfos.length; ) {
    inputTotalUSDValueE36 += openTokenInfos[i].inputAmt * tokenPriceE36s[i];
    borrowTotalUSDValueE36 += openTokenInfos[i].borrowAmt * tokenPriceE36s[i];
    unchecked {
        ++i;
    }
}

positionOpenUSDValueE36 = inputTotalUSDValueE36 + borrowTotalUSDValueE36;
netPnLE36 = positionCurUSDValueE36.toInt256() - positionOpenUSDValueE36.toInt256();
```


In [26]:
def extract_codeblocks(description):
    codeblock = ""

    writing = False

    for line in description.split("\n"):
        if line.strip().startswith("```"):
            if writing:
                yield codeblock
                codeblock = ""
                writing = False
            else:
                writing = True
            continue

        if writing:
            codeblock += line + "\n"

In [43]:
content = "function sellUnderlying"

item = descriptions[descriptions.str.contains(content)].iloc[0]

# item = descriptions.iloc[0]

# print(item)

formatted_blocks = []
for codeblock in extract_codeblocks(item):

    print("CODEBLOCK:", codeblock)
    print("====")

    prompt = fix_template.format(codeblock)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to("cuda")
    output_tokens = model.generate(
        **inputs, max_new_tokens=512, pad_token_id=tokenizer.pad_token_id
    )
    decoded_output = tokenizer.decode(
        output_tokens[0],
        skip_special_tokens=True,
        pad_token_id=tokenizer.pad_token_id,
    )

    raw_output = decoded_output.split("Formatted code block:\n")[1].strip()

    if not raw_output.startswith("```"):
        raw_output = "```\n" + raw_output
    if not raw_output.endswith("```"):
        raw_output += "\n```"

    formatted_blocks.append(raw_output)

print(len(formatted_blocks))

codeblocks = "\n".join(formatted_blocks)#.replace("\n", "\\n")
print(codeblocks)

CODEBLOCK:     function sellUnderlying(
        address u,
        uint256 m,
        uint128 a,
        uint128 s
    ) external returns (uint128) {
        // Get the pool for the market
        IPool pool = IPool(pools[u][m]);

        // Get the number of PTs received for selling `a` underlying tokens
        uint128 expected = pool.sellBasePreview(a);

        // Verify slippage does not exceed the one set by the user
        if (expected < s) {
            revert Exception(16, expected, 0, address(0), address(0));
        }

        // Transfer the underlying tokens to the pool
        Safe.transferFrom(IERC20(pool.base()), msg.sender, address(pool), a);
// rest of code
    function sellPrincipalToken(
        address u,
        uint256 m,
        uint128 a,
        uint128 s
    ) external returns (uint128) {
        // Get the pool for the market
        IPool pool = IPool(pools[u][m]);

        // Preview amount of underlying received by selling `a` PTs
        uint256 expecte

In [2]:
import pandas as pd
import csv

DIALECT_NAME = "db_dialect"
csv.register_dialect(
    DIALECT_NAME,
    delimiter=",",
    quoting=csv.QUOTE_MINIMAL,
    escapechar="\\",
)

# Output file name
output_file = 'cleaned-up-code.csv'
header = True
# Open the output file in write mode to create/overwrite it
with open(output_file, 'w') as outfile:
    # Iterate through each chunk file
    for i in range(21):
        input_file = f'../cleaned-up-code-{i}.csv'
        print(f"Processing {input_file}")

        # Read the input CSV file
        df = pd.read_csv(input_file, dialect=DIALECT_NAME)

        # For the first file, include the header
        df.to_csv(output_file, mode='a', index=False, header=header)
        header = False


Processing ../cleaned-up-code-0.csv
Processing ../cleaned-up-code-1.csv
Processing ../cleaned-up-code-2.csv
Processing ../cleaned-up-code-3.csv
Processing ../cleaned-up-code-4.csv
Processing ../cleaned-up-code-5.csv
Processing ../cleaned-up-code-6.csv
Processing ../cleaned-up-code-7.csv
Processing ../cleaned-up-code-8.csv
Processing ../cleaned-up-code-9.csv
Processing ../cleaned-up-code-10.csv
Processing ../cleaned-up-code-11.csv
Processing ../cleaned-up-code-12.csv


FileNotFoundError: [Errno 2] No such file or directory: '../cleaned-up-code-12.csv'

In [5]:

test1 = """
```\\n```\\n{\\n  \"@openzeppelin/contracts\": \"4.3.1\"\,\\n  \"@openzeppelin/test-helpers\": \"0.5.6\"\,\\n  \"@openzeppelin/contracts-upgradeable\": \"4.3.1\"\\n}\\n```\\n```
"""

test2 = """
365,"```\n{\n  ""@openzeppelin/contracts"": ""4.3.1"",\n  ""@openzeppelin/test-helpers"": ""0.5.6"",\n  ""@openzeppelin/contracts-upgradeable"": ""4.3.1""\n}\n```","The UUPSUpgradeable vulnerability is a critical severity bug discovered in OpenZeppelin's contracts, specifically in versions 4.1.0 to 4.3.1. The affected contracts include `@openzeppelin/contracts` and `@openzeppelin/contracts-upgradeable`, which are widely used in the kyber-swap contracts, as evident from the `package.json` file.\n\nThe vulnerability allows an attacker to manipulate the upgrade process of UUPSUpgradeable contracts, enabling them to execute arbitrary code during the upgrade process. This can lead to unauthorized changes to the contract's behavior, potentially resulting in the theft of funds or unauthorized access to sensitive data.\n\nThe affected contracts, `PoolOracle.sol` and `TokenPositionDescriptor.sol`, are both UUPSUpgradeable and require immediate attention to ensure the security of the kyber-swap contracts. It is essential to upgrade these contracts to a fixed version, specifically `@openzeppelin/contracts` version 4.3.2 or higher, and `@openzeppelin/contracts-upgradeable` version 4.3.2 or higher, to mitigate this vulnerability.","To mitigate the UUPSUpgradeable vulnerability in OpenZeppelin Contracts, follow these steps:nn1. **Update the OpenZeppelin library to the latest version**: Ensure that you are using the latest version of the OpenZeppelin Contracts library, which is version 4.3.2 or higher. This will ensure that you have the fix for the UUPSUpgradeable vulnerability.nn2. **Verify the UUPS implementation contracts**: Review the OpenZeppelin UUPS documentation to understand the correct implementation of UUPSUpgradeable contracts. This will help you identify any potential issues and ensure that your contracts are properly configured.nn3. **Check the affected contracts**: Identify the contracts that are affected by the UUPSUpgradeable vulnerability, such as PoolOracle.sol and TokenPositionDescriptor.sol. Review the code and ensure that they are using the latest version of the OpenZeppelin Contracts library.nn4. **Implement the UUPSUpgradeable contracts correctly**: Follow the OpenZeppelin UUPS documentation to implement the UUPSUpgradeable contracts correctly. This includes setting the `UUPSUpgradeable` contract as the base contract for your upgradeable contracts.nn5. **Test the UUPSUpgradeable contracts**: Thoroughly test the UUPSUpgradeable contracts to ensure that they are functioning correctly and that the upgrade process is working as expected.nn6. **Monitor the UUPSUpgradeable contracts**: Continuously monitor the UUPSUpgradeable contracts to ensure that they are not vulnerable to any new attacks or exploits.nnBy following these steps, you can effectively mitigate the UUPSUpgradeable vulnerability in OpenZeppelin Contracts and ensure the security of your smart contracts.",upgradeable contract vulnerability
"""


print(test1.replace('\\n', '\n'))
print(test2.replace('\\n', '\n'))


```
```
{
  "@openzeppelin/contracts": "4.3.1"\,
  "@openzeppelin/test-helpers": "0.5.6"\,
  "@openzeppelin/contracts-upgradeable": "4.3.1"
}
```
```


365,"```
{
  ""@openzeppelin/contracts"": ""4.3.1"",
  ""@openzeppelin/test-helpers"": ""0.5.6"",
  ""@openzeppelin/contracts-upgradeable"": ""4.3.1""
}
```","The UUPSUpgradeable vulnerability is a critical severity bug discovered in OpenZeppelin's contracts, specifically in versions 4.1.0 to 4.3.1. The affected contracts include `@openzeppelin/contracts` and `@openzeppelin/contracts-upgradeable`, which are widely used in the kyber-swap contracts, as evident from the `package.json` file.

The vulnerability allows an attacker to manipulate the upgrade process of UUPSUpgradeable contracts, enabling them to execute arbitrary code during the upgrade process. This can lead to unauthorized changes to the contract's behavior, potentially resulting in the theft of funds or unauthorized access to sensitive data.

The affected contracts, `PoolOr